In [30]:
import os

from dotenv import load_dotenv, dotenv_values
load_dotenv()

True

In [35]:
env_vars = dotenv_values()
for key in env_vars.keys():
    print(key)

OPENAI_API_KEY
PROXYCURL_API_KEY
TAVILY_API_KEY
LANGCHAIN_TRACING_V2
LANGCHAIN_ENDPOINT
LANGCHAIN_API_KEY
LANGCHAIN_PROJECT


# 1. OpenAI: Chat API

En este apartado vamos a se llama a la API de OpenAI directamente:

In [10]:
from openai import OpenAI

In [21]:
# Inicializar el cliente
client = OpenAI()

messages = [{"role": "user", "content": "What is 1+1?"}]
    
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
)

response

ChatCompletion(id='chatcmpl-BUZrssXxWHgbTF1e3GVIMMK3ZBtE2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1+1 equals 2.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1746627416, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=14, total_tokens=22, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [22]:
def get_completion(prompt):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.0,
    )
    
    return response.choices[0].message.content

In [23]:
response = get_completion("What is 1+1?")
response

'1+1 equals 2.'

In [26]:
customer_email = """Arrr, I be fuming that me blender lid flew off and \
splattered me kitchen walls with smoothie! And to make matters worse,\
the warranty don't cover the cost of cleaning up me kitchen. I need yer help \
right now, matey!"""

style = """American English in a calm and respectful tone"""

prompt = f"""Translate the text that is delimited by triple backticks into a \
style that is {style}.

text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone.

text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!```



In [27]:
get_completion(prompt)

"I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

# LangChain: Chat API

In [49]:
from langchain_openai import ChatOpenAI

In [50]:
chat = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f1030a966d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f1030e86790>, root_client=<openai.OpenAI object at 0x7f1030e87c10>, root_async_client=<openai.AsyncOpenAI object at 0x7f1030a96bd0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [51]:
from langchain.prompts import ChatPromptTemplate


template_string = """Translate the text that is delimited by triple backticks into a \
style that is {style}.

text: ```{customer_email}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [52]:
prompt_template

ChatPromptTemplate(input_variables=['customer_email', 'style'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['customer_email', 'style'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}.\n\ntext: ```{customer_email}```\n'), additional_kwargs={})])

In [53]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['customer_email', 'style'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}.\n\ntext: ```{customer_email}```\n')

In [54]:
prompt_template.messages[0].prompt.input_variables

['customer_email', 'style']

In [59]:
customer_email = """Arrr, I be fuming that me blender lid flew off and \
splattered me kitchen walls with smoothie! And to make matters worse,\
the warranty don't cover the cost of cleaning up me kitchen. I need yer help \
right now, matey!"""

customer_style = """American English in a calm and respectful tone"""

customer_messages = prompt_template.format_messages(
    style=customer_style,
    customer_email=customer_email
    )

In [61]:
customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone.\n\ntext: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!```\n", additional_kwargs={}, response_metadata={})]

In [60]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [ ]:
print(customer_messages[0])

In [64]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [65]:
customer_response.content

"I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."